In [2]:
import pandas as pd
import sys
sys.path.append('../src/')
%matplotlib inline
import numpy as np
from net.modules import *
from data.load_data import *
from data.data_loader import load_data , spilit_refit_test, Seq2PointDataset
import torch
from einops import rearrange
from torch import nn

In [158]:
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn
    def forward(self, x):
        return self.fn(x) + x

class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    def forward(self, x):
        return self.fn(self.norm(x))

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, dim)
        )
    def forward(self, x):
        return self.net(x)
    
    
    
class Attention(nn.Module):
    def __init__(self, dim=50, heads = 8):
        super().__init__()
        self.heads = heads
        self.scale = dim ** -0.5
        self.to_qkv = nn.Linear(dim,dim*3*heads, bias = False)
        self.to_out = nn.Linear(dim*heads, dim)
    def forward(self, x):
        b, n, _, h = *x.shape, self.heads
        qkv = self.to_qkv(x)
        q, k, v = torch.split(qkv, qkv.size(-1)//3, dim=-1)
        dots = torch.matmul(q.view(b, n, self.heads, -1), k.view(b, n, -1, self.heads)) *self.scale 
        attn = dots.softmax(dim=-1)
        out = torch.matmul(attn, q.view(b, n, self.heads, -1))     # (bs, n_heads, q_length, dim_per_head)
        out = rearrange(out, 'b c h d -> b c (h d)')
        out =  self.to_out(out)
        return out

In [159]:


model = Attention() 
x = torch.randn(4, 1, 50)
model(x)


tensor([[[ 0.1073, -0.1985,  0.1542,  0.1815, -0.1100, -0.0326,  0.2754,
           0.0006, -0.0403, -0.0490, -0.0900,  0.0306,  0.2606, -0.1323,
           0.0563, -0.0187,  0.1833, -0.1931,  0.1775,  0.0090,  0.2378,
          -0.0916,  0.0474, -0.0295,  0.1758, -0.2126,  0.0046, -0.2506,
           0.1311, -0.2287, -0.0642,  0.3006,  0.1802, -0.0861,  0.0818,
          -0.1271, -0.2213, -0.0790, -0.0996, -0.1093, -0.0216,  0.3117,
           0.1757,  0.0943, -0.0021,  0.0685, -0.0405, -0.0251, -0.1404,
           0.0457]],

        [[-0.0704,  0.0647,  0.0489,  0.0787,  0.0412,  0.0649, -0.0862,
           0.0240,  0.1134,  0.0090, -0.2821, -0.3184,  0.0333,  0.0144,
           0.0993,  0.0212, -0.0031, -0.0221, -0.1573, -0.0652, -0.1274,
          -0.0782,  0.0542,  0.1586,  0.1367, -0.1701, -0.1496,  0.0571,
           0.1322,  0.1875, -0.2084, -0.1738, -0.1797, -0.0248, -0.0302,
          -0.1544,  0.0716, -0.0808,  0.0748,  0.1718,  0.0466, -0.0147,
          -0.0671, -0.0896, -

In [169]:
class NILMiT(nn.Module):
    def __init__(self, seq_len=50, patch_size=5, num_classes=12, 
                 dim=50, depth=4, heads=8, 
                 mlp_dim=128, channels = 1, n_quantiles=3):
        super().__init__()
        assert seq_len % patch_size == 0, 'image dimensions must be divisible by the patch size'
        num_patches = (seq_len // patch_size) 
        patch_dim = channels * patch_size 
        self.patch_size = patch_size
        self.n_quantiles = n_quantiles
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.patch_to_embedding = nn.Linear(patch_dim, dim)
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.transformer = Transformer(dim, depth, heads, mlp_dim)
        self.to_cls_token = nn.Identity()
        self.mlp_head = nn.Sequential(
            nn.Linear(dim, mlp_dim),
            nn.GELU(),
            nn.Linear(mlp_dim, num_classes)
        )
        self.mlp_classifier = nn.Sequential(
            nn.Linear(dim, mlp_dim),
            nn.GELU(),
            nn.Linear(mlp_dim, num_classes*2))
        
        self.mlp_regress = nn.Sequential(
            nn.Linear(dim, mlp_dim),
            nn.GELU(),
            nn.Linear(mlp_dim, num_classes*n_quantiles))
        
    def forward(self, x):
        p = self.patch_size
        B = x.size(0)
        x  = rearrange(x.unsqueeze(-1), 'b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = p, p2 = 1)
        x  = self.patch_to_embedding(x)
        cls_tokens = self.cls_token.expand(x.shape[0], -1, -1)
        
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding
        x = self.transformer(x)
        x = self.to_cls_token(x[:, 0])
        
        states_logits   = self.mlp_classifier(x).reshape(B, 2, -1)
        power_logits    = self.mlp_regress(x)
        if self.n_quantiles>1:
            power_logits = power_logits.reshape(B, self.n_quantiles, -1)
        return states_logits, power_logits

In [170]:
model = NILMiT()
x = torch.randn(4, 1, 50)
model(x)

(tensor([[[ 0.1269, -0.1844, -0.2307,  0.0641, -0.6315,  0.2955,  0.2687,
            0.5740,  0.1166,  0.1411, -0.0677, -0.0164],
          [ 0.0956, -0.0302,  0.0058, -0.2058,  0.0164,  0.1673, -0.4383,
           -0.0162, -0.3665,  0.0420, -0.1358, -0.0666]],
 
         [[ 0.1269, -0.1844, -0.2307,  0.0641, -0.6315,  0.2955,  0.2687,
            0.5740,  0.1166,  0.1411, -0.0677, -0.0164],
          [ 0.0956, -0.0302,  0.0058, -0.2058,  0.0164,  0.1673, -0.4383,
           -0.0162, -0.3665,  0.0420, -0.1358, -0.0666]],
 
         [[ 0.1269, -0.1844, -0.2307,  0.0641, -0.6315,  0.2955,  0.2687,
            0.5740,  0.1166,  0.1411, -0.0677, -0.0164],
          [ 0.0956, -0.0302,  0.0058, -0.2058,  0.0164,  0.1673, -0.4383,
           -0.0162, -0.3665,  0.0420, -0.1358, -0.0666]],
 
         [[ 0.1269, -0.1844, -0.2307,  0.0641, -0.6315,  0.2955,  0.2687,
            0.5740,  0.1166,  0.1411, -0.0677, -0.0164],
          [ 0.0956, -0.0302,  0.0058, -0.2058,  0.0164,  0.1673, -0.4383,


## Multi-head attention